<a href="https://colab.research.google.com/github/dataview26/freecodecamp-machine-learning-with-python-projects/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# Load books
books_filename = '/content/books.csv'
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author', 'year', 'publisher', 'image_s', 'image_m', 'image_l'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

# Load ratings
ratings_filename = '/content/ratings.csv'
df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)

print("Data loaded successfully!")
print(f"Books: {df_books.shape}")
print(f"Ratings: {df_ratings.shape}")

Data loaded successfully!
Books: (271379, 3)
Ratings: (1149780, 3)


In [ ]:
# Count ratings per user
user_counts = df_ratings['user'].value_counts()
# Count ratings per book
book_counts = df_ratings['isbn'].value_counts()

# Keep only users with 200+ ratings
users_to_keep = user_counts[user_counts >= 200].index
df_ratings = df_ratings[df_ratings['user'].isin(users_to_keep)]

# Keep only books with 100+ ratings
books_to_keep = book_counts[book_counts >= 100].index
df_ratings = df_ratings[df_ratings['isbn'].isin(books_to_keep)]

print(f"After filtering:")
print(f"Ratings: {df_ratings.shape}")
print(f"Users: {df_ratings['user'].nunique()}")
print(f"Books: {df_ratings['isbn'].nunique()}")

After filtering:
Ratings: (49781, 3)
Users: 888
Books: 731


In [ ]:
# Combine books and ratings
df = pd.merge(df_ratings, df_books, on='isbn')

print(f"Merged data shape: {df.shape}")
display(df.head())

Merged data shape: (49517, 5)


,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,James Finn Garner
1,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
2,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
3,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Paperback)),Janet Evanovich
4,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff


In [ ]:
# Create matrix: rows=books, columns=users, values=ratings
df_pivot = df.pivot_table(
    index='title',
    columns='user',
    values='rating'
).fillna(0)

print(f"Pivot shape: {df_pivot.shape}")
print(f"Books: {df_pivot.shape[0]}")
print(f"Users: {df_pivot.shape[1]}")

Pivot shape: (673, 888)
Books: 673
Users: 888


In [ ]:
# Convert to sparse matrix for efficiency
mat_books_users = csr_matrix(df_pivot.values)

print(f"Matrix shape: {mat_books_users.shape}")

Matrix shape: (673, 888)


In [ ]:
# Train the model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model_knn.fit(mat_books_users)

print("Model trained successfully!")

Model trained successfully!


In [ ]:
def get_recommends(book_title):
    # Find book in dataset
    if book_title not in df_pivot.index:
        print(f"Book not found: {book_title}")
        return None

    # Get book index
    book_idx = df_pivot.index.get_loc(book_title)

    # Find 6 nearest neighbors (including the book itself)
    distances, indices = model_knn.kneighbors(
        mat_books_users[book_idx],
        n_neighbors=6
    )

    # Build recommendations list (skip first item which is the book itself)
    recommendations = []
    for i in range(1, 6):
        book_name = df_pivot.index[indices[0][i]]
        distance = distances[0][i]
        recommendations.append([book_name, distance])

    # Reverse list so closest books are at the end
    recommendations.reverse()

    return [book_title, recommendations]

print("Function created successfully!")

Function created successfully!


In [ ]:
# Test with the example book
result = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(result)

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Catch 22', np.float32(0.7939835)], ['The Witching Hour (Lives of the Mayfair Witches)', np.float32(0.74486566)], ['Interview with the Vampire', np.float32(0.73450685)], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', np.float32(0.53763384)], ['The Vampire Lestat (Vampire Chronicles, Book II)', np.float32(0.5178412)]]]


In [ ]:
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

    if recommends[0] != "The Queen of the Damned (Vampire Chronicles (Paperback))":
        test_pass = False

    recommended_books = [
        "Catch 22",
        "The Witching Hour (Lives of the Mayfair Witches)",
        "Interview with the Vampire",
        "The Tale of the Body Thief (Vampire Chronicles (Paperback))",
        "The Vampire Lestat (Vampire Chronicles, Book II)"
    ]

    recommended_books_dist = [
        0.793983519077301,
        0.7448656558990479,
        0.7345068454742432,
        0.5376338362693787,
        0.5178412199020386
    ]

    for i in range(5):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False

    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉
